# LMSYS | XGB Baseline

(original notebook: https://www.kaggle.com/code/sercanyesiloz/lmsys-xgb-baseline)

# 1. Libraries

In [ ]:
import gc
import os
import re
import numpy as np
import pandas as pd

import nltk
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

# 2. Configuration

In [ ]:
class config:
    root = "/kaggle/input/lmsys-chatbot-arena/"
    train_path = os.path.join(root, "train.csv")
    test_path = os.path.join(root, "test.csv")
    sample_submission_path = os.path.join(root, "sample_submission.csv")
    seed = 42
    n_splits = 10

# 3. Loading Data

In [ ]:
# Read the training, test, and sample submission datasets from the specified paths
train = pd.read_csv(config.train_path)
test = pd.read_csv(config.test_path)
sample_submission = pd.read_csv(config.sample_submission_path)

# If the test dataset has fewer than 10 rows, limit the training dataset to the first 10,000 rows
if test.shape[0] < 10:
    train = train.iloc[:10000]

# Define a function to process strings by removing brackets and splitting sentences
# NOTE: Another way would be to convert to JSON and then join, but this is probably most efficient in Python
def process(input_str):
    stripped_str = input_str.strip('[]')  # Remove leading and trailing square brackets
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # Split by "," and remove surrounding quotes
    return ' '.join(sentences)  # Join the sentences with a space

# Apply the `process` function to the prompt and response columns in the train dataset
train["prompt"] = train["prompt"].apply(process)
train["response_a"] = train["response_a"].apply(process)
train["response_b"] = train["response_b"].apply(process)

# Apply the `process` function to the prompt and response columns in the test dataset
test["prompt"] = test["prompt"].apply(process)
test["response_a"] = test["response_a"].apply(process)
test["response_b"] = test["response_b"].apply(process)

# Print the shapes of the train and test datasets
print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")
print("-"*90)

# Print the total number of missing values in the train and test datasets
print(f"train missing values: {train.isnull().sum().sum()}")
print(f"test missing values: {test.isnull().sum().sum()}")
print("-"*90)

# Display the first few rows of the train dataset
train.head()


# 4. Feature Engineering

In [ ]:
class Preprocessor:

    # Calculate cosine similarity between two texts
    def cosine_sim(self, text1: str, text2: str):
        try:
            vectorizer = TfidfVectorizer(ngram_range=(1, 3))  # Create a TF-IDF vectorizer (word-importance) with n-grams from 1 to 3
            vectorizer.fit([text1, text2])  # Fit the vectorizer on both texts
            output = vectorizer.transform([text1, text2]).toarray()  # Transform texts to TF-IDF vectors
            cos_sim = cosine_similarity(output)  # Calculate cosine similarity between vectors
            return cos_sim[0][1]  # Return the similarity score between text1 and text2
        except:
            print(f"cosine_sim exception with '{text1}' and '{text2}'")
            return np.nan  # Return NaN in case of an exception

    # Calculate Jaccard similarity between two texts
    def jaccard_sim(self, text1: str, text2: str):
        set1 = set(text1.split())  # Split text1 into set of words
        set2 = set(text2.split())  # Split text2 into set of words
        intersection = set1.intersection(set2)  # Find intersection of both sets
        union = set1.union(set2)  # Find union of both sets
        return len(intersection) / len(union)  # Return Jaccard similarity score
    
    # Count the number of quoted segments in a text
    def count_quotes(self, text: str) -> int:
        single_quote_pattern = r"'(.*?)'"  # Pattern for single quotes
        double_quote_pattern = r'"(.*?)"'  # Pattern for double quotes
        single_quotes = re.findall(single_quote_pattern, text)  # Find all single-quoted segments
        double_quotes = re.findall(double_quote_pattern, text)  # Find all double-quoted segments
        total_quotes = len(single_quotes) + len(double_quotes)  # Sum the counts of both types of quotes
        return total_quotes  # Return the total count of quoted segments
    
    # Count the number of new-lines in a text
    def count_new_lines(self, text: str) -> int:
        return text.count('\\n')  # Return the count of newline characters in the text
    
    # Count the number of bulleted lists in the text
    def count_bulleted_lists(self, text: str) -> int:
        bullet_pattern = r'(\\n|^)[\*\-\+]\s'  # Pattern for bulleted list items
        return len(re.findall(bullet_pattern, text))  # Return the count of bulleted list items
    
    # Tokenize text into lowercase words
    def tokenize(self, text: str):
        return nltk.word_tokenize(text.lower())

    # Generate n-grams from the tokenized text
    def generate_ngrams(self, text: str, n: int):
        tokens = self.tokenize(text)  # Tokenize the text
        return list(ngrams(tokens, n))  # Generate n-grams from tokens

    # Count overlapping n-grams between two texts
    def count_ngram_overlaps(self, text1: str, text2: str, n: int) -> int:
        try:
            ngrams1 = self.generate_ngrams(text1, n)  # Generate n-grams for text1
            ngrams2 = self.generate_ngrams(text2, n)  # Generate n-grams for text2
            counter1 = Counter(ngrams1)  # Count n-grams in text1
            counter2 = Counter(ngrams2)  # Count n-grams in text2
            overlap = counter1 & counter2  # Find the overlap between the two counters
            overlap_count = sum(overlap.values())  # Sum the counts of overlapping n-grams
            return overlap_count  # Return the overlap count
        except:
            return 0  # Return 0 in case of an exception
        
    # Run preprocessing on the data
    def run(self, data: pd.DataFrame) -> pd.DataFrame:
        
        # Calculate unigram, bigram, and trigram overlaps between response_a and response_b
        data["respa_respb_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 1), axis=1)
        data["respa_respb_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 2), axis=1)
        data["respa_respb_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 3), axis=1)

        # Calculate unigram, bigram, and trigram overlaps between response_a and prompt
        data["respa_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 1), axis=1)
        data["respa_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 2), axis=1)
        data["respa_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 3), axis=1)

        # Calculate unigram, bigram, and trigram overlaps between response_b and prompt
        data["respb_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 1), axis=1)
        data["respb_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 2), axis=1)
        data["respb_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 3), axis=1)
        
        # Calculate the length of tokenized texts
        data["respa_len"] = data["response_a"].apply(lambda x: len(self.tokenize(x)))
        data["respb_len"] = data["response_b"].apply(lambda x: len(self.tokenize(x)))
        data["prompt_len"] = data["prompt"].apply(lambda x: len(self.tokenize(x)))
        
        # Calculate length ratios between response_a, response_b, and prompt
        data["respa_prompt_len_ratio"] = data["respa_len"] / data["prompt_len"]
        data["respb_prompt_len_ratio"] = data["respb_len"] / data["prompt_len"]
        data["respa_respb_len_ratio"] = data["respa_len"] / data["respb_len"]
        
        # Calculate length differences between response_a, response_b, and prompt
        data["respa_respb_len_diff"] = data["respa_len"] - data["respb_len"]
        data["respa_prompt_len_diff"] = data["respa_len"] - data["prompt_len"]
        data["respb_prompt_len_diff"] = data["respb_len"] - data["prompt_len"]
        
        # Calculate overlap ratios for unigrams, bigrams, and trigrams between response_a and prompt
        data["respa_prompt_overlap_unigram_ratio"] = data["respa_prompt_overlap_unigram"] / data["prompt_len"]
        data["respa_prompt_overlap_bigram_ratio"] = data["respa_prompt_overlap_bigram"] / data["prompt_len"]
        data["respa_prompt_overlap_trigram_ratio"] = data["respa_prompt_overlap_trigram"] / data["prompt_len"]

        # Calculate overlap ratios for unigrams, bigrams, and trigrams between response_b and prompt
        data["respb_prompt_overlap_unigram_ratio"] = data["respb_prompt_overlap_unigram"] / data["prompt_len"]
        data["respb_prompt_overlap_bigram_ratio"] = data["respb_prompt_overlap_bigram"] / data["prompt_len"]
        data["respb_prompt_overlap_trigram_ratio"] = data["respb_prompt_overlap_trigram"] / data["prompt_len"]
        
        # Count the number of quotes in response_a, response_b, and prompt
        data["respa_quotes"] = data["response_a"].apply(lambda x: self.count_quotes(x))
        data["respb_quotes"] = data["response_b"].apply(lambda x: self.count_quotes(x))
        data["prompt_quotes"] = data["prompt"].apply(lambda x: self.count_quotes(x))

        # Count the number of new-lines in response_a, response_b, and prompt
        data["respa_new_lines"] = data["response_a"].apply(lambda x: self.count_new_lines(x))
        data["respb_new_lines"] = data["response_b"].apply(lambda x: self.count_new_lines(x))
        data["prompt_new_lines"] = data["prompt"].apply(lambda x: self.count_new_lines(x))

        # Count the number of bulleted lists in response_a, response_b, and prompt
        data["respa_bullets"] = data["response_a"].apply(lambda x: self.count_bulleted_lists(x))
        data["respb_bullets"] = data["response_b"].apply(lambda x: self.count_bulleted_lists(x))
        data["prompt_bullets"] = data["prompt"].apply(lambda x: self.count_bulleted_lists(x))
        
        # Calculate cosine and Jaccard similarities between response_a and response_b
        data["respa_respb_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["response_b"]), axis=1)
        data["respa_respb_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["response_b"]), axis=1)
        
        # Calculate cosine and Jaccard similarities between response_a and prompt
        data["respa_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["prompt"]), axis=1)
        data["respa_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["prompt"]), axis=1)
        
        # Calculate cosine and Jaccard similarities between response_b and prompt
        data["respb_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_b"], x["prompt"]), axis=1)
        data["respb_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_b"], x["prompt"]), axis=1)
        
        return data  # Return the processed dataframe


In [ ]:
%%time

preprocessor = Preprocessor()
train = preprocessor.run(train)
test = preprocessor.run(test)
train.head()

In [ ]:
# List of columns to drop from the dataset
drop_cols = ["id", "response_a", "response_b", "prompt"]

# List of target columns indicating the winner
target_cols = ["winner_model_a", "winner_model_b", "winner_tie"]

# Name of the final target column
target = "target"

# Initialize the target column with NaN values
train[target] = np.nan

# Iterate over the target columns and set the corresponding index in the target column
for idx, t in enumerate(target_cols):
    train.loc[train[t] == 1, target] = idx  # Set target column to the index where target column value is 1

# Convert the target column to integer type
train[target] = train[target].astype("int32")

# Display the first few rows of the updated dataframe
train.head()

# 5. Modeling

In [ ]:
# Drop specified columns from the training dataset and assign the result to X
X = train.drop(columns=target_cols + drop_cols + [target] + ["model_a", "model_b"], axis=1)

# Assign the target column to y
y = train[target]

# Drop specified columns from the test dataset and assign the result to X_test
X_test = test.drop(columns=drop_cols, axis=1)

# Replace infinite values (-inf and inf) with NaN in the training feature set
X = X.replace([-np.inf, np.inf], np.nan)

# Replace infinite values (-inf and inf) with NaN in the test feature set
X_test = X_test.replace([-np.inf, np.inf], np.nan)

In [ ]:
# Set up stratified cross-validation with the specified number of splits, shuffle, and seed
cv = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)

# Initialize an array to store the average predictions for the test set
test_preds = np.zeros(shape=(X_test.shape[0], y.nunique()))

# Initialize a list to store cross-validation scores (log loss) for each fold
cv_scores = list()

# Get the list of feature names
features = X.columns.tolist()

# Prepare a DataFrame to store feature importances for each fold
feat_imp_df = pd.DataFrame({"feature": features})

# Loop over each fold in the cross-validation
for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    print(f"| Fold {idx+1} |".center(90, "="))  # Print the fold number

    # Split the data into training and validation sets for the current fold
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_val, y_val = X.loc[val_idx], y.loc[val_idx]

    # Print the shapes of the training and validation sets
    print(f'train: {X_train.shape}')
    print(f'val: {X_val.shape}')
    
    # Initialize the XGBoost classifier with specified hyperparameters
    model = xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=3,
        eval_metric='mlogloss',
        subsample=0.8,
        n_estimators=650,
        learning_rate=0.045,
        max_depth=5,
        random_state=config.seed,
        device="gpu"
    )
    
    # Train the model with early stopping on the validation set
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        early_stopping_rounds=75,
        verbose=75
    )
    
    # Predict probabilities on the validation set
    val_preds = model.predict_proba(X_val)

    # Calculate the log loss for the validation set
    val_log_loss = log_loss(y_val, val_preds, eps="auto")
    print(f"val log loss: {val_log_loss:.5f}")

    # Append the log loss to the list of cross-validation scores
    cv_scores.append(val_log_loss)
    
    # Update test predictions with the current fold's predictions, averaged over all folds
    test_preds += model.predict_proba(X_test) / cv.get_n_splits()
    
    # Merge the current fold's feature importances into the DataFrame
    feat_imp_df = feat_imp_df.merge(
        pd.DataFrame(
            {
                "feature": features,
                f"fold_{idx+1}_feat_imp": model.feature_importances_,
            }
        ),
        on=["feature"],
        how="left",
    )

# Print a separator line and the average cross-validated log loss
print("="*90)
print(f"CV: {np.mean(cv_scores):.5f}")

# Calculate the average feature importance across all folds
feat_imp_df["avg_importance"] = feat_imp_df.iloc[:, 1:].mean(axis=1)

# Plot the top 50 features by average importance
plt.figure(figsize=(12, 10))
sns.barplot(
    data=feat_imp_df.sort_values(by="avg_importance", ascending=False).iloc[
        :50
    ],
    x="avg_importance",
    y="feature",
    color="royalblue",
    width=0.75,
)
plt.title("Average Feature Importances for All Folds", size=12)
plt.show()

# 6. Saving Submission

In [ ]:
for idx, t in enumerate(target_cols):
    sample_submission[t] = test_preds[:, idx]
sample_submission.head()

In [ ]:
sample_submission.to_csv("submission.csv", index=False)